In [1]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/data/tungtx2/tmp/'
import numpy as np
import pdb
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer

In [2]:
df_rs_summed = pd.read_csv('resume_summarized.csv')
df_rs_summed.head(2)

,Unnamed: 0,Resume_str,Category,Resume,Resume_no_stem,clean,clean_str,clean_no_stem,clean_str_no_stem,resume_summarized
0,0,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,HR,hr administr market associ hr administr summar...,hr administrator marketing associate hr admini...,"['hr', 'administr', 'market', 'associ', 'hr', ...",hr administr market associ hr administr summar...,"['hr', 'administrator', 'marketing', 'associat...",hr administrator marketing associate hr admini...,Hilton is a customer service manager with year...
1,1,"HR SPECIALIST, US HR OPERATIONS ...",HR,hr specialist us hr oper summari versatil medi...,hr specialist us hr operations summary versati...,"['hr', 'specialist', 'hr', 'oper', 'summari', ...",hr specialist hr oper summari versatil media p...,"['hr', 'specialist', 'hr', 'operations', 'summ...",hr specialist hr operations summary versatile ...,Hr specialist hr operations company city state...


In [3]:
df_jd_summed = pd.read_csv('jd_summarized.csv')
df_jd_summed.head(2)

,job_title,job_description,list_tags,contractor_tier,job_ids,jd_cleaned,job_info,jd_summarized
0,Tiktok followers and video likes,I opened an account and I’m trying to get it v...,"Advertising, Graphic Design, YouTube Developme...",Intermediate,0,opened account trying viral followers likes da...,Intermediate-Tiktok followers and video likes-...,A new Tiktok account was created to attract in...
1,Advisory work re google ads,We are looking for an experienced professional...,"Social Media Advertising, Social Media Managem...",Expert,54,looking experienced professional provide advis...,Expert-Advisory work re google ads-looking exp...,Experienced professional is looking for an exp...


# Demo summarize text

In [4]:
from transformers import pipeline

summarizer = pipeline("summarization", model="philschmid/bart-large-cnn-samsum")

In [18]:
text = df_rs_summed.iloc[0]['clean_str_no_stem'].strip()
summed_text = summarizer(text)[0]['summary_text']
print('ORIGINAL RESUME: ', text)
print('SUMMARIZED RESUME: ', summed_text)

ORIGINAL RESUME:  hr administrator marketing associate hr administrator summary dedicated customer service manager years experience hospitality customer service management respected builder leader customer focused teams strives instill shared enthusiastic commitment customer service highlights focused customer satisfaction team management marketing savvy conflict resolution techniques training development skilled multi tasker client relations specialist accomplishments missouri dot supervisor training certification certified ihg customer loyalty marketing segment hilton worldwide general manager training certification accomplished trainer cross server hospitality systems hilton onq micros opera pms fidelio opera reservation ors holidex completed courses seminars customer service sales strategies inventory control loss prevention safety time management leadership performance assessment experience hr administrator marketing associate hr administrator dec current company city state helps 

# Demo text2vec

In [19]:
from sentence_transformers import SentenceTransformer

minilm_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [21]:
emb = minilm_model.encode(summed_text)
print(emb.shape)

(384,)


# Demo Model

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import pytorch_lightning as pl
import torchmetrics
from copy import deepcopy
import pdb

class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size) -> None:
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [29]:
ckpt_path = 'ckpt/exp2/epoch=14-train_loss=0.051-val_loss=0.150-train_acc=0.959-val_acc=0.804.ckpt'
state = torch.load(ckpt_path, map_location='cpu')['state_dict']
for k in list(state.keys()):
    state[k.replace('model.', '')] = state.pop(k)

model = Net(
    input_size=384,
    hidden_size=384,
    output_size=384
)
model.load_state_dict(state)
print('Loaded model successfully')

Loaded model successfully


In [30]:
rs_rows = df_rs_summed.sample(3)
ls_rs = list(rs_rows['resume_summarized'])
print(ls_rs)

['Stock management professional years experience working leading retailers. Stock clerk years experience coordinating receipt processing products. Aviation supply specialist company city state tracked inventory and conducted cycle counts audits. Aircraft care aide company city State created a safe environment preventing falls accidents and provided excellent service attention to customers. ', 'Director information technology is a senior project manager with years of experience in the health care private sector, government and non-profit sectors. He has mentored hundreds of project managers in his career. He is also a principal consultant with the company crm practice. ', "Senior interior designer is a team player and multitasking expert. He consults clients and works as a drafter for his company City State. He is a member of Asid allied member and an iida associate member. He's a certified beginner golf expert and a skier."]


In [31]:
jd_rows = df_jd_summed.sample(20)
ls_jd = list(jd_rows['job_info'])
print(ls_jd)

['Intermediate-Data Extraction of Key Contacts - Investment Profile-project goal create universe financial asset managers alternative investments real estate investments infrastructure investments companies provide software asset managers companies identifying key titles relevant contact information total contacts maximum marketing titles company', 'Intermediate-Organize my LEGAL data ,intergrate into my automated personal BOT using the best in AI GPT4 ,python-organize digital data emails set landing page click funnel setup emails mailchimp assist integrating private association platform barter bank time bank wallet type programs integrate monetized platform products data needs scraped organized converted digit digital products', 'Intermediate-Structural Engineer Needed for Permit Building Plans (600sqft Aircrete Dome)-hello seeking competent efficient draftsman complete building plans county zoning permit requires load bearing calculations dome walls plan view complete dimensioned fou

In [55]:
ls_rs = [
    'Iam a hr specialist',
    'I am a Software developer with 10 years of exp',
]

ls_jd = [
    'we are hiring people with experience in human resource',
    'we need to hire a software developer that require technical skills'
]

In [61]:
def cosine_sim(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)


rs = ls_rs[0]
rs_emb = minilm_model.encode(rs)
with torch.no_grad():
    v1 = model(torch.from_numpy(rs_emb).unsqueeze(0))
v1 = v1.cpu().numpy()[0]

scores, jds = [], []
for jd in ls_jd:
    jd_emb = minilm_model.encode(jd)
    with torch.no_grad():
        v2 = model(torch.from_numpy(jd_emb).unsqueeze(0))
    v2 = v2.cpu().numpy()[0]
    sim = cosine_sim(v1, v2)
    # sim = cosine_sim(rs_emb, jd_emb)
    scores.append(sim)
    jds.append(jd)

max_idx = np.argmax(scores)
print('RESUME: ', rs)
print()
print('MOST SIMILAR JD: ', jds[max_idx])
print()
print('SCORE: ', scores)

RESUME:  Iam a hr specialist

MOST SIMILAR JD:  we are hiring people with experience in human resource

SCORE:  [0.8310306, 0.6282274]


In [34]:
v1.shape

torch.Size([1, 384])

# test on test sets

In [62]:
import pickle

with open('test_pairs.pkl', 'rb') as f:
    test_pairs = pickle.load(f)

In [64]:
pair = test_pairs[0]
print(pair.keys())

dict_keys(['resume', 'job_info', 'job_id', 'resume_summarized', 'jd_summarized', 'rs_emb', 'jd_emb', 'is_match'])


In [69]:
threshold = 0.4
match_pred, match_true = [], []
for i in range(len(test_pairs)):
    pair = test_pairs[i]
    rs_emb = pair['rs_emb']
    jd_emb = pair['jd_emb']

    with torch.no_grad():
        v1 = model(torch.from_numpy(rs_emb).unsqueeze(0))[0]
        v2 = model(torch.from_numpy(jd_emb).unsqueeze(0))[0]

    v1 = v1.cpu().numpy()
    v2 = v2.cpu().numpy()
    sim = cosine_sim(v1, v2)
    print('cosine sim: '.upper(), sim)
    print('is match: '.upper(), pair['is_match'])
    print()
    
    match_pred.append(int(sim>threshold))
    match_true.append(int(pair['is_match']))

num_true = len([i for i in range(len(match_true)) if match_pred[i]==match_true[i]])
print('Acc: ', num_true/len(match_pred))

COSINE SIM:  0.055986676
IS MATCH:  0

COSINE SIM:  0.03069271
IS MATCH:  0

COSINE SIM:  0.25467443
IS MATCH:  0

COSINE SIM:  0.08562795
IS MATCH:  0

COSINE SIM:  0.2041213
IS MATCH:  0

COSINE SIM:  0.012463317
IS MATCH:  0

COSINE SIM:  0.19983622
IS MATCH:  0

COSINE SIM:  0.5038887
IS MATCH:  1

COSINE SIM:  0.077567734
IS MATCH:  0

COSINE SIM:  0.37716544
IS MATCH:  0

COSINE SIM:  0.055438116
IS MATCH:  0

COSINE SIM:  0.09298951
IS MATCH:  0

COSINE SIM:  -0.106456876
IS MATCH:  0

COSINE SIM:  0.4442692
IS MATCH:  1

COSINE SIM:  0.27785748
IS MATCH:  0

COSINE SIM:  0.085965656
IS MATCH:  0

COSINE SIM:  0.42893112
IS MATCH:  0

COSINE SIM:  0.55796814
IS MATCH:  1

COSINE SIM:  0.4833593
IS MATCH:  0

COSINE SIM:  0.7404061
IS MATCH:  1

COSINE SIM:  0.7060707
IS MATCH:  1

COSINE SIM:  0.6092553
IS MATCH:  1

COSINE SIM:  0.6335446
IS MATCH:  1

COSINE SIM:  0.8080211
IS MATCH:  1

COSINE SIM:  0.50585884
IS MATCH:  0

COSINE SIM:  -0.022581952
IS MATCH:  0

COSINE SIM: 

In [70]:
# test some pair
pair = test_pairs[-4]
print(pair['resume_summarized'])
print()
print(pair['jd_summarized'])
print()

print(match_pred[-4])
print(match_true[-4])

Customer service representative March current company city state established new customer accounts including checking savings lines credit loans in April. Company city state interviewed prospective clients to learn financial needs and discuss existing coverage. The company opened new customers accounts in March. March company  opened new accounts in April and July. 

We need to hire a Google Sheets Expert for Creating Ultimate Monthly Budget Planner Template. The job description is as follows. We are looking for reliable detail oriented detail oriented individual work on a long-term basis. We need someone with strong proficiency in google sheets spreadsheet design, attention to detail, accuracy and data entry calculations.

1
1
